# 金、银分析

当前数据集包括期货主力合约数据，上海现货数据。当前策略主要是金银比在历史极值附近的回归策略。--20181123

## 需要引入的库

In [1]:
#-*-coding:utf-8 -*-
import pandas as pd
import numpy as np
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from pyecharts import Line, Overlap, Grid

# 数据所在文件夹
future代表期货数据，
spot代表现货数据，
sh代表上海市场，
london代表伦敦市场。数据维护由其他程序承担，本分析程序仅进行分析

In [ ]:
gold_future = "C:\\quanttime\\data\\gold\\sh_future\\gold.csv"
silver_future = "C:\\quanttime\\data\\gold\\sh_future\\silver.csv"
gold_spot = "C:\\quanttime\\data\\gold\\sh_spot\\gold.csv"
silver_spot = "C:\\quanttime\\data\\gold\\sh_spot\\silver.csv"

In [ ]:
gold_future_data = pd.read_csv(gold_future,parse_dates=["date"], index_col=["date"])
print("gold future data before duplicated:%d" % len(gold_future_data))
gold_future_data = gold_future_data[~gold_future_data.reset_index().duplicated().values]
print("gold future data after duplicated:%d" % len(gold_future_data))
silver_future_data = pd.read_csv(silver_future, parse_dates=["date"],index_col=["date"])
silver_future_data = silver_future_data[~silver_future_data.reset_index().duplicated().values]

gold_spot_data = pd.read_csv(gold_spot, index_col=["date"])
gold_spot_data.index = pd.to_datetime(gold_spot_data.index)
silver_spot_data = pd.read_csv(silver_spot, index_col=["date"])
silver_spot_data.index = pd.to_datetime(silver_spot_data.index)

future_data = pd.merge(gold_future_data,silver_future_data,left_index=True,right_index=True,suffixes=('_gold','_silver'))
spot_data = pd.merge(gold_spot_data, silver_spot_data, left_index=True, right_index=True)
#spot_data["price_gold"] = pd.to_numeric(spot_data["price_gold"],downcast='float')
#spot_data["price_silver"] = pd.to_numeric(spot_data["price_silver"],downcast='float')

print(type(gold_spot_data.index))
print(type(silver_spot_data.index))


In [ ]:
future_data.tail(5)

In [ ]:
spot_data.head(5)

In [ ]:
spot_data["price_silver"]=pd.to_numeric(spot_data["price_silver"])
print(type(spot_data.iloc[0,0]))
print(type(spot_data.iloc[0,1]))

In [ ]:
#del nan value
future_data = future_data.dropna()
spot_data = spot_data.dropna()
#print(data.head(5))

future_data["compare"] = future_data["close_gold"]/future_data["close_silver"]*1000
spot_data["compare"] = spot_data["price_gold"]/spot_data["price_silver"]*1000
spot_data["price_silver"] = spot_data["price_silver"]/10 #将银价按100g为单位计价，主要是与黄金价格在同一范围
spot_data = spot_data.round(2)

In [ ]:
future_data_gold = np.array(future_data.loc[:, ["close_gold"]])
future_data_silver = np.array(future_data.loc[:,["close_silver"]])
future_data_compare = np.array(future_data.loc[: ,["compare"]])
future_datetime = np.array(future_data.index.map(str))


-----------------分界线--------------------
以下部分为图线，以上部分为数据准备

1、黄金期货主力合约图

In [ ]:
line_future_gold = Line("future gold")
line_future_gold.add("future gold",future_datetime,future_data_gold.round(1),\
                     mark_point=["average","max","min"],\
                     is_datazoom_show=True,\
                    mark_point_symbol=None,\
                    yaxis_min=150,yaxis_max=300)
line_future_gold

2、白银期货主力合约图

In [ ]:
line_future_silver = Line("future silver")
line_future_silver.add("future silver",future_datetime,future_data_silver,mark_point=["average","max","min"],is_datazoom_show=True)
line_future_silver

3、期货主力合约金银比及对比数据

In [ ]:
line_future = Line("future AU/AG")
line_future.add("future gold",future_datetime,future_data_gold.round(1),\
                     mark_point=["average","max","min"]
                     )
line_future.add("future silver",future_datetime,future_data_silver,mark_point=["average","max","min"],is_datazoom_show=True,\
               datazoom_xaxis_index=[0,1],legend_top="50%")


line_future_compare = Line("future compare",title_top="50%")
line_future_compare.add("future AU/AG",future_datetime,future_data_compare.round(1),mark_point=["average","max","min"],is_datazoom_show=True,
                       yaxis_min=40,yaxis_max=85)

grid_future = Grid()
grid_future.add(line_future,grid_top="60%")
grid_future.add(line_future_compare,grid_bottom="60%")

4、期货主力合约金银比统计数据

In [ ]:
future_data.loc[: ,["compare"]].describe()

In [ ]:
future_data[["close_gold","close_silver","compare"]].sort_values(by=['compare'],ascending=False)

前一次金银比回归分析。前一次金银比极值出现在2016-02-29	gold：259.40	silver：3312.0	AU/AG：78.321256

In [ ]:
future_data_2016 = future_data.loc["2016-02-29":,["close_gold","close_silver","compare"]]
'''
取出从2016-2-29开始的后，第一个小于compare<70的索引
future_data_2016[future_data_2016.compare<70].index
'''
end = future_data_2016[future_data_2016.compare<70].index[0]

    

In [ ]:
future_data_2016 = future_data.loc["2016-02-29":end, ["close_gold","close_silver","compare"]]
future_data_2016

在该时间段如果按照极值78回落到70区间，采用做空gold，做多相同价值合约的silver，区间获利计算如下：

In [ ]:
gold_open = future_data.loc["2016-02-29",["close_gold"]]#gold 开仓价
gold_close = future_data.loc[end,["close_gold"]]#silver 结算价

silver_open = future_data.loc["2016-02-29",["close_silver"]]
silver_close = future_data.loc[end,["close_silver"]]

gold_total_value = gold_open.iloc[0] * 1000
silver_amount = round(gold_total_value /(silver_open.iloc[0] * 15))
silver_total_value = silver_open.iloc[0] * 15 * silver_amount

gold_return =  gold_total_value - gold_close.iloc[0] * 1000
silver_return =  silver_close.iloc[0] * 15 * silver_amount - silver_total_value
fact_return = silver_return - ( -gold_return)

print("===============================")
print("gold 开仓价格: %r, 平仓价格:%r"%(gold_open.iloc[0],gold_close.iloc[0]))
print("gold(short)total profit: %r"%gold_return)
print("silver 开仓价格: %r, 平仓价格:%r"%(silver_open.iloc[0],silver_close.iloc[0],))
print("silver(long) total profit: %r"%silver_return)
print("total return: %r"%fact_return)
print("total contract value:gold(%r),silver(%r)"%(gold_total_value.round(2),silver_total_value))
print("time consume: 2016-02-29-%r"%(end))
print("===============================")

期货主力合约，金银比回归策略，期间最大回撤

In [ ]:
future_data_2016["tmp_loss"] =  (gold_open.iloc[0] - future_data_2016["close_gold"]) * 1000 + \
                               (future_data_2016["close_silver"] - silver_open.iloc[0]) * 15 * silver_amount

loss_datetime = np.array(future_data_2016.index.map(str))
tmp_loss_value = np.array(future_data_2016.loc[:,["tmp_loss"]])
line_loss = Line("loss")
line_loss.add("loss",loss_datetime,tmp_loss_value.round(2),mark_point=["average","max","min"],is_stack=True)
line_loss

In [ ]:
future_data_20 = future_data.iloc[(len(future_data)-20):len(future_data)]

In [ ]:
future_data_20.loc[:,["compare"]].describe()

In [ ]:
v_5 = future_data_20.loc[:,["compare"]].quantile(0.05)
future_data_20.loc["5%"] = v_5


In [ ]:
print(future_data)
analyse_data = future_data.loc["2018/11/23":"2019/1/21",["compare"]]
analyse_data.describe()

# 现货金银比分析

In [ ]:
spot_data_gold = np.array(spot_data.loc[:,["price_gold"]])
spot_data_silver = np.array(spot_data.loc[:, ["price_silver"]])
spot_data_compare = np.array(spot_data.loc[:, ["compare"]])
spot_datatime = np.array(spot_data.index.map(str))

line_spot = Line("gold_sliver",title_top="50%")
line_spot_compare = Line("AU/AG")

line_spot.add("gold", spot_datatime, spot_data_gold, mark_point=["average","max","min"],legend_top="50%",is_stack=True,is_datazoom_show=True,\
               datazoom_xaxis_index=[0,1])
line_spot.add("silver",spot_datatime,spot_data_silver,mark_point=["average","max","min"],legend_top="50%",is_stack=True)
line_spot_compare.add("AU/AG",spot_datatime,spot_data_compare,mark_point=["average","max","min"],is_datazoom_show=True)

grid = Grid()
grid.add(line_spot,grid_top="60%")
#grid.add(line_silver,grid_top=300)
grid.add(line_spot_compare,grid_bottom="60%")


现货金银比的统计规律

In [ ]:
spot_data["compare"].describe()

In [ ]:
spot_data[["price_gold","price_silver","compare"]].sort_values(by=['compare'],ascending=False)

期货主力合约与现货，2016-3月段，金银比比较

In [ ]:
future_data_201603 = future_data.loc["2016/2/15":"2016-12-31",["close_gold","close_silver","compare"]]
spot_data_201603 = spot_data.loc["2016-02-15":"2016-12-31",["price_gold","price_silver","compare"]]
compare_201603 = pd.merge(future_data_201603,spot_data_201603,left_index=True,right_index=True,suffixes=["_future","_spot"])
datatime_201603 = np.array(compare_201603.index.map(str))
compare_future_201603 = np.array(compare_201603.loc[:,["compare_future"]])
compare_spot_201603 = np.array(compare_201603.loc[:,["compare_spot"]])

line_compare_201603 = Line("compare_201603")
line_compare_201603.add("compare_future",datatime_201603,compare_future_201603.round(2),mark_point=["average","max","min"],\
                        is_stack=True,is_datazoom_show=True,yaxis_min=30,yaxis_max=90)
line_compare_201603.add("compare_spot",datatime_201603,compare_spot_201603,mark_point=["average","max","min"],\
                        is_datazoom_show=True,yaxis_min=30,yaxis_max=90)
line_compare_201603

In [ ]:
gold_future_mins = "C:\\quanttime\\data\\gold\\sh_future\\gold_mins.csv"
silver_future_mins = "C:\\quanttime\\data\\gold\\sh_future\\silver_mins.csv"
gold_future_mins_data = pd.read_csv(gold_future_mins, index_col=["datetime"])
print("gold future mins data before duplicated:%d" % len(gold_future_mins_data))
gold_future_mins_data = gold_future_mins_data[~gold_future_mins_data.reset_index().duplicated().values]
print("gold future mins data after duplicated:%d" % len(gold_future_mins_data))
silver_future_mins_data = pd.read_csv(silver_future_mins, index_col=["datetime"])
silver_future_mins_data = silver_future_mins_data[~silver_future_mins_data.reset_index().duplicated().values]

future_data_mins = pd.merge(gold_future_mins_data,silver_future_mins_data,left_index=True,right_index=True,suffixes=('_gold','_silver'))
#del nan value
future_data_mins = future_data_mins.dropna()
future_data_mins["compare"] = future_data_mins["close_gold"]/future_data_mins["close_silver"]*1000
print("future_data_mins:%d"%len(future_data_mins))

In [ ]:
future_data_mins["compare"].describe()

In [ ]:
def get_appoint_date(df,strDate,nDays):
    '''
    获取指定日期往前推或者往后推的几天，输入df是一个包含日期类型的dataframe
    df:pandas dataFrame,index需要是str 类型的 date/datetime
    strDate:input 日期，如2010-10-10
    nDays：往前或者往后的n天,<0 往前推，>0往后推
    ''' 
    #先去重，才能保证前移或者后退的天数准确
    if not isinstance(df,pd.DataFrame):
        print("input df type is not pd.DataFrame")
        return -1
    
    df = df[~df.reset_index().duplicated().values]
    if nDays < 0:
        if df.index.tolist().index(strDate) + nDays >= 0:
            pre_date = df.index[df.index.tolist().index(strDate) + nDays]
            return pre_date
        else:
            return -1
    else:
        if df.index.tolist().index(strDate) + nDays <= len(df.index)-1:
            after_date = df.index[df.index.tolist().index(strDate) + nDays]
            return after_date
        else:
            return -1

In [ ]:
get_appoint_date(future_data,"2019/3/9",-10)

In [ ]:
tmp = future_data
isinstance(tmp,dict)

In [ ]:
def testAUAGRadio(df,strTradeDate,position):
    '''
    df:pandas dataFrame,index需要是str 类型的 date/datetime
    strTradeDate:str类交易日
    position：dict，仓位信息
    '''
    #if strEnd == None:
    #    strEnd = (datetime.today().date()-timedelta(days=1)).strftime('%Y-%m-%d')
    if not isinstance(df,pd.DataFrame):
        print("input df type is not pd.DataFrame")
        return -1
    preDate = get_appoint_date(df,strTradeDate,20)
    if preDate == -1:
        print("获取交易日：%s前20天的日期失败")
        return -1
    
    
    
    

In [ ]:
gold_future = "C:\\quanttime\\data\\gold\\sh_future\\gold.csv"
silver_future = "C:\\quanttime\\data\\gold\\sh_future\\silver.csv"

stander_dtype = {'open': float, "close": float, "high": float, "low": float, "volume":float, "money":float  }
gold_future_data = pd.read_csv(gold_future, parse_dates=["date"], index_col=["date"], dtype=stander_dtype)
gold_future_data = gold_future_data[~gold_future_data.reset_index().duplicated().values]

silver_future_data = pd.read_csv(silver_future, parse_dates=["date"], index_col=["date"], dtype=stander_dtype)
silver_future_data = silver_future_data[~silver_future_data.reset_index().duplicated().values]

future_data = pd.merge(gold_future_data, silver_future_data, left_index=True, right_index=True,
                               suffixes=('_gold', '_silver'))
future_data["compare"] = future_data["close_gold"] / future_data["close_silver"] * 1000
future_data.head()

In [ ]:
gold_future_mins = "C:\\quanttime\\data\\gold\\sh_future\\gold_mins.csv"
silver_future_mins = "C:\\quanttime\\data\\gold\\sh_future\\silver_mins.csv"
gold_future_mins_data = pd.read_csv(gold_future_mins, parse_dates=["datetime"], index_col=["datetime"], dtype=stander_dtype)
#print("gold future mins data before duplicated:%d" % len(self.gold_future_mins_data))
gold_future_mins_data = gold_future_mins_data[~gold_future_mins_data.reset_index().duplicated().values]
#print("gold future mins data after duplicated:%d" % len(self.gold_future_mins_data))
silver_future_mins_data = pd.read_csv(silver_future_mins, parse_dates=["datetime"], index_col=["datetime"], dtype=stander_dtype)
silver_future_mins_data = silver_future_mins_data[~silver_future_mins_data.reset_index().duplicated().values]

future_data_mins = pd.merge(gold_future_mins_data, silver_future_mins_data, left_index=True, right_index=True,
                                    suffixes=('_gold', '_silver'))
future_data_mins["compare"] = future_data_mins["close_gold"] / future_data_mins["close_silver"] * 1000

In [ ]:
pd.set_option('max_row', 2000)
future_data_mins.loc["2016-04-13 14:00:00":"2016-04-25 14:00:00",["close_gold","close_silver","compare"]]

In [ ]:

tmp = future_data_mins.index[-1].date() 
future_data.loc[tmp]

In [ ]:
future_data_mins.loc[pd.to_datetime("2013-03-16 09:01:00"):pd.to_datetime("2013-03-16 23:59:00")]

In [ ]:
pd.to_datetime("2019-03-13 09:00:00")

In [ ]:
def get_appoint_date_stat_info(future_data, strDate, nDays):
    '''
    获取指定日期前推或者往后推的几天的统计信息，统计信息来源为合并后的self.future_data
    :param strDate:input 日期，如2010-10-10
    :param nDays:往前或者往后的n天,<0 往前推，>0往后推
    :return:统计信息的pd.dataframe

    eg, describe:
                       compare
             count	20.000000
             mean	78.479471
              std	0.625898
              min	77.456493
             25%	77.980835
             50%	78.401895
             75%	79.030748
             max	79.437169
    '''
    inputDate = pd.to_datetime(strDate)
    columns_name = ["count", "mean", "std", "min", "25%", "50%", "75%", "max"]
    df_empty = pd.DataFrame(columns=columns_name)
    if nDays < 0:
        if future_data.index.tolist().index(inputDate) + nDays >= 0:
            pre_date = future_data.index[
                future_data.index.tolist().index(inputDate) + nDays]
            pre_date_1 = future_data.index[
                future_data.index.tolist().index(inputDate) - 1]
            #print("input date: %s, stat begin:%s, end:%s" % (strDate, pre_date, pre_date_1))
            return future_data.loc[pre_date:pre_date_1, ["compare"]].describe()
        else:
            return df_empty
    else:
        if future_data.index.tolist().index(inputDate) + nDays <= len(
                future_data.index) - 1:
            after_date = future_data.index[
                future_data.index.tolist().index(inputDate) + nDays]
            after_date_1 = future_data.index[
                future_data.index.tolist().index(inputDate) + 1]
            return future_data.loc[after_date_1:after_date, ["compare"
                                                             ]].describe()
        else:
            return df_empty

In [ ]:
gold_future_data = pd.read_csv(gold_future,parse_dates=["date"], index_col=["date"])
gold_future_data = gold_future_data[~gold_future_data.reset_index().duplicated().values]
silver_future_data = pd.read_csv(silver_future, parse_dates=["date"],index_col=["date"])
silver_future_data = silver_future_data[~silver_future_data.reset_index().duplicated().values]
future_data = pd.merge(gold_future_data,silver_future_data,left_index=True,right_index=True,suffixes=('_gold','_silver'))
future_data = future_data.dropna()
future_data["compare"] = future_data["close_gold"] / future_data["close_silver"] * 1000    
future_date_trade_date = future_data.index
columns_name = ["date","count", "mean", "std", "min", "25%", "50%", "75%", "max"]
#df_describe = pd.DataFrame(columns=columns_name)
#df_describe.index_name = "date"
data_list = []

for trade_date in future_date_trade_date:
    tmp_list = []
    df_stat = get_appoint_date_stat_info(future_data,trade_date,-20)
    if df_stat.empty:
        continue
    else:
        tmp_list = [trade_date, df_stat.loc["count",["compare"]].compare, df_stat.loc["mean",["compare"]].compare, \
                    df_stat.loc["std",["compare"]].compare,df_stat.loc["min",["compare"]].compare, df_stat.loc["25%",["compare"]].compare,\
                   df_stat.loc["50%",["compare"]].compare, df_stat.loc["75%",["compare"]].compare,\
                   df_stat.loc["max",["compare"]].compare]
        data_list.append(tmp_list)
df_describe = pd.DataFrame(data=data_list,columns=columns_name)
df_describe      


In [ ]:
df_stat.loc["count",["compare"]].compare

In [8]:
gold_future = "C:\\quanttime\\data\\gold\\sh_future\\gold.csv"
silver_future = "C:\\quanttime\\data\\gold\\sh_future\\silver.csv"

stander_dtype = {'open': float, "close": float, "high": float, "low": float, "volume": float, "money": float}
gold_future_data = pd.read_csv(gold_future, parse_dates=["date"], index_col=["date"],
                                            dtype=stander_dtype)
gold_future_data = gold_future_data[~gold_future_data.reset_index().duplicated().values]

silver_future_data = pd.read_csv(silver_future, parse_dates=["date"], index_col=["date"],
                                              dtype=stander_dtype)
silver_future_data = silver_future_data[~silver_future_data.reset_index().duplicated().values]

future_data = pd.merge(gold_future_data, silver_future_data, left_index=True, right_index=True,
                                    suffixes=('_gold', '_silver'))
future_data["compare"] = future_data["close_gold"] / future_data["close_silver"] * 1000
# 去重
future_data = future_data.dropna()
future_data_trade_date = future_data.index

today = datetime.today().date()
data_last_date = future_data.index[-1]
print(today.strftime("%Y-%m-%d"))
print(data_last_date.strftime("%Y-%m-%d"))

df_stat_20 = future_data.iloc[-20:]
df_stat = df_stat_20.loc[:, ["compare"]].describe()
df_stat

2019-03-28
2019-03-26


,compare
count,20.000000
mean,78.922580
std,0.450431
min,77.845200
25%,78.732472
50%,79.038267
75%,79.244646
max,79.437169


In [10]:
df_stat.loc["max",["compare"]].compare

79.43716912789078